In [ ]:
#| default_exp datasets.quantum_exp

# Libraries

In [1]:
#| export
import numpy as np
from scipy.stats import unitary_group

from fastai.vision.all import DataLoaders

import torch
from torch.utils.data import TensorDataset, DataLoader as DataLoader_torch

import itertools
from functools import reduce
from itertools import product

In [18]:
from nbdev import nbdev_export
nbdev_export() # Export the notebook

# State tomography class

In [2]:
#| export
class StateTomography(object):
    """
    Creates various datasets corresponding to local and global measurements on
    random mixed multi-qubit quantum states.

    The input data is a `num_datapoints` times `num_measure` matrix where each
    data point consists of `num_measure` measurements on a random `num_qubits`-
    qubit density matrix.

    Actions are all Pauli measurements and `num_random_actions` non-local and local 
    measurements, in total say, M measurements. The action data is a
    `num_datapoint` times M matrix of measurement results on the random pauli
    matrices of the input data. 

    The measurement bases for the generation of the input data can be saved in an
    appropriate file, as well as the measurement bases of the actions.

    The random density matrices can be saved in another appropriate file.

    We can also save a hypothesis data file where specific local Pauli operators
    are saved.

    Args:
        num_qubits (int): Number of qubits.
        num_measure (int): Number of different measurements performed on each qubit.
        num_random_actions (int): Number of additional random actions.
        num_datapoints (int): Number of datapoints collected for each measurement set.
    """
    def __init__(self, num_qubits=2, num_measure=75, num_random_actions=20, num_datapoints=100000, test_data_points=100):
        self.num_qubits = num_qubits
        self.num_measure = num_measure
        self.num_random_actions = num_random_actions
        assert (self.num_random_actions/(self.num_qubits+1)).is_integer()
        self.num_datapoints = num_datapoints
        
        self.measurements = self._create_measurements()

        self.paulis = [np.eye(2, dtype=complex), 
                       np.array([[0,1],[1,0]],dtype=complex), # X
                       np.array([[0,-1j],[1j,0]],dtype=complex), # Y
                       np.array([[1,0],[0,-1]],dtype=complex) # Z
                       ]
        
        self.num_actions = len(self.paulis)**self.num_qubits-1+self.num_random_actions
        self.num_test_datapoints=test_data_points ##########
        self.create_dataset()
        self.get_test_data()
        # self.data = DataHandler(self.num_qubits+1, datasets=raw_data)

    def create_dataset(self):
        """
        Creates one input dataset corresponding to a full tomography and one 
        action dataset corresponding to the expectation value of one measurement per action.
        Also keeps track of the density matrices that are randomly generated for this data.
        """
        # input data
        self.input_data = np.empty((self.num_datapoints, self.num_measure))
        self.true_data = np.empty((self.num_datapoints, 2**self.num_qubits, 2**self.num_qubits), dtype=complex)
        for l in range(self.num_datapoints):
            rho = self._random_mixed_state()
            
        
            self.input_data[l] = self._create_datapoint(rho)
            self.true_data[l] = rho
            # generate input data
        
        # action performed on data
        self.action_measures = np.empty((len(self.paulis)**self.num_qubits-1+self.num_random_actions, 2**self.num_qubits, 2**self.num_qubits),dtype=complex) #TODO
        self.action_data = np.empty((self.num_datapoints, 
                                     len(self.paulis)**self.num_qubits-1+self.num_random_actions,
                                     1))
        
        for i, rho in enumerate(self.true_data):
            j = 0
            for action in product(range(len(self.paulis)), repeat=self.num_qubits):
                if action==tuple(0 for i in range(self.num_qubits)):
                    continue
                # define projector from paulis
                multi_op = [np.eye(2) for k in range(self.num_qubits)]
                for index, a in enumerate(action):
                    multi_op[index] = self.paulis[a]
                proj = 1/2*(reduce(np.kron, [np.eye(2) for k in range(self.num_qubits)]) + reduce(np.kron, multi_op))
                self.action_data[i, j] = self._measure(rho, proj)
                # print(self.action_data[i,j])
                # save projector
                if i==0:
                    self.action_measures[j] = proj
                j+=1
            # add random projectors
            if i==0:
                # define random measurements
                proj_set = self._create_measurements()[0:int(self.num_random_actions/(self.num_qubits+1))]
                for s in range(self.num_qubits):
                    proj_set = np.append(proj_set, self._create_measurements(local=s)[0:int(self.num_random_actions/(self.num_qubits+1))], axis=0)
                self.action_measures[j:len(self.action_measures)] = proj_set
            for p in self.action_measures[j:len(self.action_measures)]:
                self.action_data[i, j] = self._measure(rho, p)
                j+=1


    def get_test_data(self):

        pauli_mats = [np.array([[0,1],[1,0]], dtype=complex), # pauli-x
                              np.array([[0,-1.j],[1.j,0]], dtype=complex), # pauli-y
                              np.array([[1,0],[0,-1]], dtype=complex) # pauli-z
                             ]
        axis = 3 # x,y,z
        num_datapoint = 11 # -1., -0.8, ..., 0.8, 1.
        data = torch.empty((self.num_qubits, axis, num_datapoint, self.num_measure * (self.num_qubits + 1)))
        
        
        self.datapoints = torch.empty((2**(2*self.num_qubits)-1, num_datapoint, self.num_measure))#### should be 2**(2*self.num_qubits)-1 
        
        # generate data: for each qubit, one density matrix with variable amplitudes of one pauli matrix in Bloch rep
        
        for idxa, action in enumerate(itertools.product(range(len(self.paulis)), repeat=self.num_qubits)):
        
        # for a, action in enumerate(range(len(self.paulis)), repeat=self.num_qubits):
            if idxa!=0:
        
                if action==tuple(0 for i in range(self.num_qubits)):
                    continue
                # define densities from paulis
                multi_op = [np.eye(2) for k in range(self.num_qubits)]
                for index, a in enumerate(action):
                    multi_op[index] = self.paulis[a]
        
                for ind,r in enumerate(np.arange(-1., 1.2, 0.2)):
                    rho = 1/4*(reduce(np.kron, [np.eye(2) for k in range(self.num_qubits)]) + r*reduce(np.kron, multi_op))
                    self.datapoints[idxa-1,ind] = self._create_datapoint(rho)
    
    
    # return datapoints
    
                
    def get_training_data(self, repeat = False):
        ''' 
        Outputs the data in the correct shape for the AE training. To understand the shapes:
        N: num_datapoints   |   K: num_measures   |   A: num_actions
        As the data is created in such a way that for each state, all actions are applied, the
        input is create by repeating A times the K measures over the N states. Then, we have the following
        shapes:
        - Input: if repeat: N x A x K ; else: N x K
        - Output: N x A
        '''


        if repeat:
            return np.repeat(self.input_data[:, np.newaxis, :], self.num_actions, axis=1), self.action_data.squeeze(axis = -1)
        else:
            return self.input_data, self.action_data.squeeze(axis = -1), self.action_measures#, self.datapoints
    

    def save_truth(self, file_name):
        """
        Saves the true state underlying the data.
        """
        torch.save(torch.from_numpy(self.true_data), 'data/' + file_name + '.pth')

    def save_measurements(self, file_name):
        """
        Saves the measurement bases that were used to generate the data.
        """
        torch.save(torch.from_numpy(self.measurements), 'data/' + file_name + '_input.pth')
        torch.save(torch.from_numpy(self.action_measures), 'data/' + file_name + '_actions.pth')

    def save_data(self, file_name):
        """
        Saves the input and action data separately.
        """
        torch.save(torch.from_numpy(self.input_data), 'data/' + file_name + '_input.pth')
        torch.save(torch.from_numpy(self.action_data), 'data/' + file_name + '_action.pth')

    
    def save_hypothesis(self):
        """
        Creates and saves data for hypothesis testing of a representation.

        For each qubit we generate a density matrix

        ..math::
            \rho = \frac{1}{2}(I + \vec{a}\vec{\sigma})

        and set :math:`\vec{a}=(x,0,0),(0,y,0),(0,0,z)` where we vary x,y,z
        between -1 and 1 in 0.2 steps.
        """
        # pauli matrices for Bloch representation
        pauli_mats = [np.array([[0,1],[1,0]], dtype=complex), # pauli-x
                      np.array([[0,-1.j],[1.j,0]], dtype=complex), # pauli-y
                      np.array([[1,0],[0,-1]], dtype=complex) # pauli-z
                     ]
        axis = 3 # x,y,z
        num_datapoint = 11 # -1., -0.8, ..., 0.8, 1.
        data = torch.empty((self.num_qubits, axis, num_datapoint, self.num_measure * (self.num_qubits + 1)))
        # generate data: for each qubit, one density matrix with variable amplitudes of one pauli matrix in Bloch rep
        for q in range(self.num_qubits):
            for axis, pauli in enumerate(self.paulis[1:]):
                for index, var in enumerate(np.arange(-1., 1.2, 0.2)):
                    # create test density matrix
                    rho = [1/2*np.eye(2) for q in range(self.num_qubits)]
                    local_rho = 1/2* (np.eye(2) + var * pauli)
                    rho[q] = local_rho
                    rho = reduce(np.kron, rho)

                    # create datapoint
                    datapoint = self._create_datapoint(rho)
                    # restructure datapoint
                    local_measure = torch.zeros(self.num_measure * (self.num_qubits+1))
                    local_measure[self.num_measure * q:self.num_measure * (q+1)] = datapoint[q]
                    #add data
                    data[q,axis,index] = local_measure

        # return datapoint
        # save data
        torch.save(data, 'data/' + file_name + '.pth')
    
    # ----------------- helper methods -----------------------------------------

    # def test_data(self):
        

    def _create_datapoint(self, rho):
        """
        Creates one datapoint consisting of measurement results for some 
        pre-specified bases on a input density matrix. 

        Args:
            rho (np.ndarray): Input density matrix.

        Returns:
            datapoint (torch.Tensor): Datapoint of size (#measurements)
        """
        # empty datapoint. For each set of local and global projection, we collect a datapoint.
        datapoint = torch.empty((self.num_measure))
        
        # collect collective measurements
        for j in range(self.num_measure):
            datapoint[j] = self._measure(rho, self.measurements[j])

        return datapoint

    def _measure(self, rho, projection):
        """
        Measures the density matrix `rho` with `projection`.

        Returns: 
            result (float): Probabiltiy to measure `projection`.
        """
        result = np.trace(rho@projection)
        
        assert(np.imag(result) < 1e-3)
        return np.real(result)

    def _create_measurements(self, local=None):
        """
        Creates the set of all measurements for the state tomography on the input state.

        Returns:
            measurements (np.ndarray): The projections that form the measurements.
        """
        measurements = np.empty((self.num_measure, 2**self.num_qubits, 2**self.num_qubits),dtype=complex)

        if local==None:
            for j in range(self.num_measure):
                state = unitary_group.rvs(2**self.num_qubits)[:, 0]
                
                proj = np.outer(state.conj(), state)
                measurements[j] = proj
        else:
            for j in range(self.num_measure):
                state = unitary_group.rvs(2)[:, 0]
                
                proj = np.outer(state.conj(), state)
                full_proj = [np.eye(2) for q in range(self.num_qubits)]
                full_proj[local] = proj
                measurements[j] = reduce(np.kron, full_proj)
        
        return measurements

    def _random_mixed_state(self):
        """
        Draws a random mixed state as the partial trace of a pure state.

        Returns:
            mixed_rho (np.ndarray): The random mixed state.
        """
        pure_state = unitary_group.rvs(2 * 2**self.num_qubits)[:, 0]
        pure_rho = np.outer(pure_state.conj(), pure_state)
        mixed_rho = np.trace(
                             pure_rho.reshape(
                                              2**self.num_qubits, 
                                              2, 
                                              2**self.num_qubits, 
                                              2), 
                             axis1=1, 
                             axis2=3
                            )
        return mixed_rho
    
    def _acted_qbit(self):
        ''' Returns a list to know, for the Pauli gates, to which qbit they acted
        0 means to none | 3 means to both
        '''
        
        acted = [0]
        for idx, action in enumerate(product(range(len(self.paulis)), repeat=self.num_qubits)):
            if idx == 0:
                continue
                
            if action[0] == 0:
                acted.append(1)
            elif action[1] == 0:
                acted.append(2)
            else:
                acted.append(3)           

        return acted
        

# Training dataset generator

In [13]:
#| export
def generate_quantum_dataset(tomography, batch_size, device = 'cuda' if torch.cuda.is_available() else 'cpu'):

    # Generate the dataset
    inputs, outputs, actions = tomography.get_training_data()

    # Action representatio 
    complex_act = torch.tensor(actions)

    # Separate real and imaginary parts for each (4, 4) matrix in the batch
    real_part = complex_act.real  # Shape: (24, 4, 4)
    imag_part = complex_act.imag  # Shape: (24, 4, 4)

    # Flatten each (4, 4) matrix into (16,) for real and imaginary parts
    real_flattened = real_part.view(15, -1)  # Shape: (24, 16)
    imag_flattened = imag_part.view(15, -1)  # Shape: (24, 16)

    # Stack real and imaginary parts along the last dimension to get shape (24, 32)
    action_rep = torch.cat([real_flattened, imag_flattened], dim=1).to(torch.float)  # Shape: (24, 32)

    # Repeat actions and states to stack them
    inputs_r = torch.repeat_interleave(torch.tensor(inputs, dtype = torch.float), 
                                        repeats = tomography.num_actions,
                                        dim = 0)

    action_r = action_rep.repeat(tomography.num_datapoints, 1)

    # Merge the inputs and actions
    merged_input = torch.hstack((inputs_r, action_r))

    # Create torch / fastai dataloaders
    train_size = int(merged_input.shape[0]*0.8)

    dataset = TensorDataset(merged_input[:train_size].to(device), 
                            torch.tensor(outputs.flatten()[:train_size], dtype = torch.float).to(device))
    loader = DataLoader_torch(dataset, batch_size=batch_size, shuffle = True)

    dataset_test = TensorDataset(merged_input[train_size:].to(device), 
                                torch.tensor(outputs.flatten()[train_size:], dtype = torch.float).to(device))
    loader_test = DataLoader_torch(dataset_test, batch_size = 2000, shuffle=True)

    return DataLoaders(loader, loader_test), loader

In [14]:
tomography = StateTomography(num_qubits=2,
                             num_measure = 75, 
                             num_random_actions= 0,
                             num_datapoints = 10) 

data, loader = generate_quantum_dataset(tomography, batch_size=2)

/tmp/ipykernel_258171/1988250523.py:64: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  self.input_data[l] = self._create_datapoint(rho)


# Test dataset $\rho(r)$ generator

In [11]:
#| export
def get_test_data(tomography, num_datapoint = 100):        
    
    datapoints = torch.empty((2**(2*tomography.num_qubits)-1, num_datapoint, tomography.num_measure))
    
    rho_points = torch.empty((2**(2*tomography.num_qubits)-1, num_datapoint, 2**(tomography.num_qubits),2**(tomography.num_qubits)))

    # generate data: for each qubit, one density matrix with variable amplitudes of one pauli matrix in Bloch rep
    
    for idxa, action in enumerate(itertools.product(range(len(tomography.paulis)), repeat=tomography.num_qubits)):
        
        if idxa!=0:
    
            if action==tuple(0 for i in range(tomography.num_qubits)):
                continue
            # define densities from paulis
            multi_op = [np.eye(2) for k in range(tomography.num_qubits)]
            for index, a in enumerate(action):
                multi_op[index] = tomography.paulis[a]
    
            for ind, r in enumerate(np.linspace(-1., 1, num_datapoint)):
                rho = 1/4*(reduce(np.kron, [np.eye(2) for k in range(tomography.num_qubits)]) + r*reduce(np.kron, multi_op))
                datapoints[idxa-1,ind] = tomography._create_datapoint(rho)
                
                rho_points[idxa-1,ind] = torch.tensor(rho)
    
    
    return datapoints

In [16]:
tomography = StateTomography(num_qubits = 2,
                             num_measure = 75, 
                             num_random_actions = 0,
                             num_datapoints = 10)

test_data = get_test_data(tomography, num_datapoint = 5)
test_data.shape

torch.Size([15, 5, 75])